# Análisis Descriptivo: Mapping Review IA y ML en Educación Matemática K-12\n
\n
**MQ1: ¿Cuántos estudios se han publicado a lo largo de los años?**\n
\n
Este notebook realiza un análisis descriptivo completo del corpus de publicaciones sobre IA y ML en educación matemática K-12.

## 1. Configuración del Entorno

In [ ]:
# Instalación de dependencias\n
!pip install pandas numpy matplotlib seaborn plotly wordcloud nltk scikit-learn textblob

In [ ]:
# Importación de librerías\n
import pandas as pd\n
import numpy as np\n
import matplotlib.pyplot as plt\n
import seaborn as sns\n
import plotly.express as px\n
import plotly.graph_objects as go\n
from plotly.subplots import make_subplots\n
from wordcloud import WordCloud\n
import warnings\n
warnings.filterwarnings('ignore')\n
\n
# Configuración de estilo\n
plt.style.use('seaborn-v0_8')\n
sns.set_palette(\"husl\")\n
plt.rcParams['figure.figsize'] = (12, 8)\n
plt.rcParams['font.size'] = 12\n
\n
# Configuración para mostrar todas las columnas\n
pd.set_option('display.max_columns', None)\n
pd.set_option('display.max_colwidth', None)

## 2. Carga de Datos desde GitHub

In [ ]:
# Cargar el dataset desde GitHub\n
# IMPORTANTE: Cambiar la URL por tu repositorio real\n
url = \"https://raw.githubusercontent.com/TU_USUARIO/TU_REPOSITORIO/main/MappingReview.csv\"\n
df = pd.read_csv(url, sep=';', encoding='utf-8')\n
\n
print(f\"Dataset cargado: {df.shape[0]} filas y {df.shape[1]} columnas\")\n
print(\"\\nPrimeras 5 filas:\")\n
df.head()

In [ ]:
# Información básica del dataset\n
print(\"Información del dataset:\")\n
print(df.info())\n
\n
print(\"\\nEstadísticas descriptivas:\")\n
print(df.describe())\n
\n
print(\"\\nValores únicos por columna:\")\n
for col in df.columns:\n
    print(f\"{col}: {df[col].nunique()} valores únicos\")

## 3. Limpieza y Preparación de Datos

In [ ]:
# Limpieza de datos\n
# Convertir Year a numérico\n
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')\n
\n
# Limpiar columnas de presencia en bases de datos\n
presence_columns = ['Present in Eric', 'Present in IEEE', 'Present in Scopus', 'Present in WoS']\n
for col in presence_columns:\n
    df[col] = df[col].fillna('No')\n
    df[col] = df[col].str.strip()\n
\n
print(\"Datos limpiados exitosamente\")\n
print(f\"Rango de años: {df['Year'].min()} - {df['Year'].max()}\")

## 4. Análisis Temporal (MQ1)

In [ ]:
# Análisis de publicaciones por año\n
yearly_publications = df['Year'].value_counts().sort_index()\n
\n
# Gráfico de líneas temporal\n
plt.figure(figsize=(15, 8))\n
plt.plot(yearly_publications.index, yearly_publications.values, marker='o', linewidth=3, markersize=8)\n
plt.fill_between(yearly_publications.index, yearly_publications.values, alpha=0.3)\n
plt.title('Evolución de Publicaciones sobre IA y ML en Educación Matemática K-12 (2020-2025)', \n
          fontsize=16, fontweight='bold')\n
plt.xlabel('Año', fontsize=14)\n
plt.ylabel('Número de Publicaciones', fontsize=14)\n
plt.grid(True, alpha=0.3)\n
plt.xticks(yearly_publications.index, rotation=45)\n
\n
# Agregar valores en los puntos\n
for i, v in enumerate(yearly_publications.values):\n
    plt.text(yearly_publications.index[i], v + 0.5, str(v), ha='center', va='bottom', fontweight='bold')\n
\n
plt.tight_layout()\n
plt.show()\n
\n
print(f\"Total de publicaciones: {len(df)}\")\n
print(f\"Año con más publicaciones: {yearly_publications.idxmax()} ({yearly_publications.max()} publicaciones)\")\n
print(f\"Año con menos publicaciones: {yearly_publications.idxmin()} ({yearly_publications.min()} publicaciones)\")

In [ ]:
# Gráfico interactivo con Plotly\n
fig = px.line(x=yearly_publications.index, y=yearly_publications.values,\n
              title='Evolución Temporal de Publicaciones',\n
              labels={'x': 'Año', 'y': 'Número de Publicaciones'})\n
\n
fig.update_traces(mode='lines+markers', line=dict(width=3), marker=dict(size=8))\n
fig.update_layout(\n
    title_font_size=16,\n
    xaxis_title_font_size=14,\n
    yaxis_title_font_size=14,\n
    hovermode='x unified'\n
)\n
\n
fig.show()

## 5. Estadísticas Descriptivas Generales

In [ ]:
# Estadísticas generales\n
print(\"=== ESTADÍSTICAS DESCRIPTIVAS GENERALES ===\\n\")\n
\n
print(f\"Total de publicaciones analizadas: {len(df)}\")\n
print(f\"Período de análisis: {df['Year'].min()} - {df['Year'].max()}\")\n
print(f\"Promedio de publicaciones por año: {len(df) / (df['Year'].max() - df['Year'].min() + 1):.1f}\")\n
\n
# Análisis por tipo de publicación\n
print(\"\\n=== DISTRIBUCIÓN POR TIPO DE PUBLICACIÓN ===\")\n
type_dist = df['Type of Publication'].value_counts()\n
print(type_dist)\n
\n
# Gráfico de tipos de publicación\n
plt.figure(figsize=(12, 8))\n
colors = plt.cm.Set3(np.linspace(0, 1, len(type_dist)))\n
plt.pie(type_dist.values, labels=type_dist.index, autopct='%1.1f%%', colors=colors, startangle=90)\n
plt.title('Distribución por Tipo de Publicación', fontsize=16, fontweight='bold')\n
plt.axis('equal')\n
plt.show()

In [ ]:
# Análisis de presencia en bases de datos\n
print(\"\\n=== PRESENCIA EN BASES DE DATOS ===\")\n
presence_summary = df[presence_columns].apply(lambda x: x.value_counts())\n
print(presence_summary)\n
\n
# Gráfico de barras para presencia en bases de datos\n
fig, axes = plt.subplots(2, 2, figsize=(15, 10))\n
axes = axes.ravel()\n
\n
for i, col in enumerate(presence_columns):\n
    counts = df[col].value_counts()\n
    axes[i].bar(counts.index, counts.values, color=['#2E8B57', '#FF6B6B'])\n
    axes[i].set_title(f'Presencia en {col.split(\"Present in \")[1]}', fontweight='bold')\n
    axes[i].set_ylabel('Número de Publicaciones')\n
    \n
    # Agregar valores en las barras\n
    for j, v in enumerate(counts.values):\n
        axes[i].text(j, v + 0.5, str(v), ha='center', va='bottom', fontweight='bold')\n
\n
plt.tight_layout()\n
plt.show()

## 6. Análisis de Tendencias

In [ ]:
# Análisis de tendencias por año y tipo de publicación\n
trend_analysis = df.groupby(['Year', 'Type of Publication']).size().unstack(fill_value=0)\n
\n
plt.figure(figsize=(15, 8))\n
trend_analysis.plot(kind='bar', stacked=True, figsize=(15, 8))\n
plt.title('Tendencias de Publicación por Año y Tipo', fontsize=16, fontweight='bold')\n
plt.xlabel('Año', fontsize=14)\n
plt.ylabel('Número de Publicaciones', fontsize=14)\n
plt.legend(title='Tipo de Publicación', bbox_to_anchor=(1.05, 1), loc='upper left')\n
plt.xticks(rotation=45)\n
plt.tight_layout()\n
plt.show()\n
\n
print(\"\\nTendencias por año y tipo de publicación:\")\n
print(trend_analysis)

## 7. Resumen y Conclusiones

In [ ]:
# Generar resumen ejecutivo\n
print(\"=== RESUMEN EJECUTIVO ===\\n\")\n
\n
print(f\"📊 Total de publicaciones analizadas: {len(df)}\")\n
print(f\"📅 Período de análisis: {df['Year'].min()} - {df['Year'].max()}\")\n
print(f\"📈 Año con mayor actividad: {yearly_publications.idxmax()} ({yearly_publications.max()} publicaciones)\")\n
print(f\"📉 Año con menor actividad: {yearly_publications.idxmin()} ({yearly_publications.min()} publicaciones)\")\n
print(f\"📊 Promedio anual: {yearly_publications.mean():.1f} publicaciones\")\n
\n
# Tipo más común\n
most_common_type = type_dist.index[0]\n
print(f\"📝 Tipo de publicación más común: {most_common_type} ({type_dist.iloc[0]} publicaciones)\")\n
\n
# Base de datos más utilizada\n
most_used_db = presence_summary.loc['Yes'].idxmax()\n
print(f\"🔍 Base de datos más utilizada: {most_used_db.split('Present in ')[1]}\")\n
\n
print(\"\\n=== CONCLUSIONES ===\")\n
print(\"1. El campo de IA y ML en educación matemática K-12 está en crecimiento\")\n
print(\"2. La mayoría de publicaciones son artículos de investigación\")\n
print(\"3. Hay una tendencia creciente en los últimos años\")\n
print(\"4. Se utilizan múltiples bases de datos para la búsqueda\")